In [132]:
import pandas as pd
from ics import Calendar, Event, DisplayAlarm
from datetime import datetime, timedelta

# read scraped output
df = pd.read_csv("output.csv", names=["url","artist_country","artist","location","date_string","insta","spotify"])
df = df.set_index('artist')

# read ratings and comments from google sheet
SHEET_ID = '1uu3_S_e2zR5O2cTbknIUh4bG1GymBtdwieFwZynTPRY'
SHEET_NAME = 'transform'
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df_ratings = pd.read_csv(url,usecols=['🧑‍🎤 artist 👩‍🎤','⭐ Mikael rating','⭐ Line rating','💭 comment'])

# rename
df_ratings = df_ratings.rename(columns={'🧑‍🎤 artist 👩‍🎤':'artist','⭐ Mikael rating':'m_rating','⭐ Line rating':'l_rating','💭 comment':'comment'})
df_ratings = df_ratings.set_index('artist')
df_ratings[['comment', 'm_rating','l_rating']] = df_ratings[['comment', 'm_rating','l_rating']].fillna(' ')



In [133]:
# join website info with ratings  
df = df.join(df_ratings, how='outer', lsuffix='_left', rsuffix='_right')


In [ ]:
# get starttime, fill nulls with timestamp 13:37
df['start_time'] = df['date_string'].str.extract(r'(\d{2}:\d{2})').fillna('13:37')


# strip irrelevant locale information; could probably be made redundant
df['new_date_string'] = df['date_string'].replace(to_replace=r'.*dag (\d{1,2})\. (\w{3})\w', value=r'\1\2', regex=True)

# convert to timestamp w timezone; ics package needs utc input
df['start_datetime'] = pd.to_datetime(df['new_date_string'] + df['start_time'], format='%d%b %Y%H:%M').dt.tz_localize('CET')

## todo handle time of day for ical. Example below
df[['date_string','start_datetime']][135:].head()

In [136]:
#df['location'].unique()
stages = {
    'Orange': '🟠 Orange',
    'Arena': '🎪 Arena',
    'Avalon': '🏝️ Avalon',
    'TBA': '❔ TBA',
    'Platform': 'Platform',
    'Gloria': '🏝️ Gloria',
    'Apollo': '🚀 Apollo',
    'Festival Site': '🏟️ Festival Site',
    'Campsite': '⛺ Campsite',
    'Destroy Me Once, Destroy Me Twice': '☠️ Destroy Me Once, Destroy Me Twice',
    'Flokkr': '📣 Flokkr',
    'Food Is Now': '🥫 Food Is Now'}

df['location'] = df['location'].map(stages)

In [ ]:
# write most important concerts to ical
c = Calendar()
for index, row in df[(df['m_rating'] == '🤯JA!') | (df['m_rating'] == '👍Ja' )].iterrows():
    e = Event()

    abbreviated_comment = ' | ' + row['comment'] if len(row['comment']) > 1 else ''
    line_rating = ' | L: ' + row['l_rating'][0] if len(row['l_rating']) > 1 else ''

    e.name = row['m_rating'][0] + " " + index + line_rating
    e.begin = row['start_datetime']
    e.end = row['start_datetime'] + timedelta(hours=1) # assume all concerts are 1 hour
    # e.description = row['comment']
    e.location = row['location'] + abbreviated_comment
    a=DisplayAlarm()
    a.trigger=timedelta(hours=-1)
    e.alarms=[a]
    c.events.add(e)

# c.events
with open('most_important.ics', 'w') as my_file:
    my_file.writelines(c.serialize_iter())
